In [1]:
# import dependencies
import pandas as pd
import pymongo

In [2]:
# loaded first dataset from csv file
vg_df = pd.read_csv("../ETL_Project/vgsales-12-4-2019-short.csv")

In [4]:
# convert Publisher to uppercase for easy comparison to second dataset
vg_df.Publisher = vg_df.Publisher.str.upper()
vg_df

,Rank,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
0,1,Wii Sports,Sports,E,Wii,NINTENDO,Nintendo EAD,7.7,NaN,82.86,NaN,NaN,NaN,NaN,NaN,2006.0
1,2,Super Mario Bros.,Platform,NaN,NES,NINTENDO,Nintendo EAD,10.0,NaN,40.24,NaN,NaN,NaN,NaN,NaN,1985.0
2,3,Mario Kart Wii,Racing,E,Wii,NINTENDO,Nintendo EAD,8.2,9.1,37.14,NaN,NaN,NaN,NaN,NaN,2008.0
3,4,PlayerUnknown's Battlegrounds,Shooter,NaN,PC,PUBG CORPORATION,PUBG Corporation,NaN,NaN,36.60,NaN,NaN,NaN,NaN,NaN,2017.0
4,5,Wii Sports Resort,Sports,E,Wii,NINTENDO,Nintendo EAD,8.0,8.8,33.09,NaN,NaN,NaN,NaN,NaN,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55787,55788,Indivisible,Role-Playing,NaN,PC,505 GAMES,Lab Zero Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
55788,55789,Lost Ember,Adventure,RP,PC,MOONEYE STUDIOS,Mooneye Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
55789,55790,Lost Ember,Adventure,RP,PS4,MOONEYE STUDIOS,Mooneye Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
55790,55791,Lost Ember,Adventure,RP,XOne,MOONEYE STUDIOS,Mooneye Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0


In [11]:
# establish connection to mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.videogame_db

In [34]:
# create a dictionary from videogame dataset dropping nulls
vg_df.reset_index(inplace=True)

data_dict = [ {k:v for k,v in m.items() if pd.notnull(v)} for m in vg_df.to_dict(orient='rows')]
data_dict

[{'index': 0,
  'Rank': 1,
  'Name': 'Wii Sports',
  'Genre': 'Sports',
  'ESRB_Rating': 'E',
  'Platform': 'Wii',
  'Publisher': 'NINTENDO',
  'Developer': 'Nintendo EAD',
  'Critic_Score': 7.7,
  'Total_Shipped': 82.86,
  'Year': 2006.0},
 {'index': 1,
  'Rank': 2,
  'Name': 'Super Mario Bros.',
  'Genre': 'Platform',
  'Platform': 'NES',
  'Publisher': 'NINTENDO',
  'Developer': 'Nintendo EAD',
  'Critic_Score': 10.0,
  'Total_Shipped': 40.24,
  'Year': 1985.0},
 {'index': 2,
  'Rank': 3,
  'Name': 'Mario Kart Wii',
  'Genre': 'Racing',
  'ESRB_Rating': 'E',
  'Platform': 'Wii',
  'Publisher': 'NINTENDO',
  'Developer': 'Nintendo EAD',
  'Critic_Score': 8.2,
  'User_Score': 9.1,
  'Total_Shipped': 37.14,
  'Year': 2008.0},
 {'index': 3,
  'Rank': 4,
  'Name': "PlayerUnknown's Battlegrounds",
  'Genre': 'Shooter',
  'Platform': 'PC',
  'Publisher': 'PUBG CORPORATION',
  'Developer': 'PUBG Corporation',
  'Total_Shipped': 36.6,
  'Year': 2017.0},
 {'index': 4,
  'Rank': 5,
  'Name': '

In [35]:
# drop games collection
db.games.drop()

In [36]:
# insert videogames dictionary to games collection
db.games.insert_many(data_dict)

In [37]:
# url for second dataset from web
url = 'https://en.wikipedia.org/wiki/List_of_video_game_publishers'

In [38]:
# retrieving table for second dataset
tables = pd.read_html(url)

In [39]:
# found correct table to use
df = tables[1]
df.head()

,Publisher,Location,Est.,Notable games published,Notes
0,07th Expansion,Japan,2002,Higurashi When They Cry Umineko When They Cry ...,NaN
1,11 bit studios,"Warsaw, Poland",2010,Frostpunk This War of Mine Moonlighter,"video game developer, marketer, and distributor"
2,1C Company,"Moscow, Russia",1991,Il-2 seriesMen of War series,specializes in localization for Russia
3,2K Games,"Novato, California, United States",2005,NBA 2K,video game marketer and distributor; subsidiar...
4,3D Realms,"Garland, Texas, United States",1987,Duke Nukem series,video game developer; also known as Apogee Sof...


In [40]:
# convert Publisher to uppercase for easy comparison to second dataset
df_upper = df.Publisher.str.upper()
df.Publisher = df_upper

In [41]:
# renaming columns due to special characters not being allowed in mongo
df_new = df[["Publisher","Location","Est."]]
df_new = df_new.rename(columns={"Location":"Publisher Location", "Est.":"Established"})

In [42]:
# create a dictionary from publisher dataset dropping nulls
df_new.reset_index(inplace=True)

data_dict = [ {k:v for k,v in m.items() if pd.notnull(v)} for m in df_new.to_dict(orient='rows')]
data_dict

C:\Users\sysco\anaconda3\lib\site-packages\pandas\core\frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{'index': 0,
  'Publisher': '07TH EXPANSION',
  'Location': 'Japan',
  'Established': 2002},
 {'index': 1,
  'Publisher': '11 BIT STUDIOS',
  'Location': 'Warsaw, Poland',
  'Established': 2010},
 {'index': 2,
  'Publisher': '1C COMPANY',
  'Location': 'Moscow, Russia',
  'Established': 1991},
 {'index': 3,
  'Publisher': '2K GAMES',
  'Location': 'Novato, California, United States',
  'Established': 2005},
 {'index': 4,
  'Publisher': '3D REALMS',
  'Location': 'Garland, Texas, United States',
  'Established': 1987},
 {'index': 5,
  'Publisher': 'THE 3DO COMPANY',
  'Location': 'Redwood City, California, United States',
  'Established': 1991},
 {'index': 6,
  'Publisher': '505 GAMES',
  'Location': 'Milan, Italy',
  'Established': 2006},
 {'index': 7,
  'Publisher': '7TH LEVEL',
  'Location': 'Dallas, Texas, United States',
  'Established': 1993},
 {'index': 8,
  'Publisher': 'A&F SOFTWARE',
  'Location': 'Rochdale, UK',
  'Established': 1981},
 {'index': 9,
  'Publisher': 'AACKOSOFT

In [45]:
# inserted publisher information into games collection
count = 0
for x in data_dict:
    db.games.update_many({"Publisher": x["Publisher"]}, {"$set": x})
    count+=1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
